# установка и импорт библиотек

In [33]:
!pip install -q datasets transformers peft bitsandbytes accelerate trl jsonlines zstandard

import os
import torch
import jsonlines
import pandas as pd

import logging
logging.basicConfig(level=logging.INFO)

from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer


## установка констант и проверка использование cuda

In [34]:
# Настройки обучения
OUTPUT_DIR = "qwen1.5-7b-chat-ru-turbo-saiga"
MODEL_NAME = "Qwen/Qwen1.5-7B-Chat"
MAX_SEQ_LENGTH = 2048
LEARNING_RATE = 2e-4
NUM_TRAIN_EPOCHS = 3
BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 8

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Конвертор исходного датасета в chatml (требуется для qwen1.5-7b)

In [35]:
from datasets import load_dataset

# Импорт и использование функции преобразования
def convert_to_chatml(example):
    """Преобразует пример в формат ChatML для Qwen"""
    formatted_text = "<|im_start|>system\nYou are a helpful assistant that accurately answers user queries in Russian.<|im_end|>\n"
    
    for role, content in zip(example['messages']['role'], example['messages']['content']):
        # Замена ролей
        if role == "bot":
            role = "assistant"
        elif role == "user":
            role = "user"
            
        # Добавление сообщения в ChatML формате
        formatted_text += f"<|im_start|>{role}\n{content}<|im_end|>\n"
    
    return {"text": formatted_text}

# Загрузка датасета
dataset = load_dataset("IlyaGusev/ru_turbo_saiga", split="train")

# Применение преобразования и сбор в массив
processed_dataset = dataset.map(convert_to_chatml)
formatted_samples = [entry['text'] for entry in processed_dataset]

# Пример первого элемента массива
print("Sample formatted entry:", formatted_samples[0])

# Проверяем размер полученного массива
print(f"Total formatted entries: {len(formatted_samples)}")


Sample formatted entry: <|im_start|>system
You are a helpful assistant that accurately answers user queries in Russian.<|im_end|>
<|im_start|>user
Мне нужно найти название фильма. В нем детей из детдомов набирают в группу для спец операций. Ты можешь мне помочь?<|im_end|>
<|im_start|>assistant
Конечно. Навскидку я могу предложить несколько фильмов. Но, скорее всего, ты ищешь фильм под названием "Сиротский батальон" (англ. "The Little Bastards"). Эту картина, снятая в 2003 году, рассказывает о группе детей совершающих различные военные миссии, посланные на линию фронта.<|im_end|>
<|im_start|>user
Да, спасибо. Извини, не было времени на поиск поисковиком. А есть ли другие фильмы на эту тему?<|im_end|>
<|im_start|>assistant
Конечно. Есть еще один фильм, который может заинтересовать тебя. Он называется "Солдаты неудачи" (англ. "Toy Soldiers"). В этом фильме подростки из интернатной школы похищаются террористами, и вынуждены сражаться за свою жизнь. Этот фильм был выпущен в 1991 году.<|im_e

In [36]:
formatted_samples[2]

'<|im_start|>system\nYou are a helpful assistant that accurately answers user queries in Russian.<|im_end|>\n<|im_start|>user\nЯ хотел бы узнать больше о том, как использовать счётчики для проверки результата.<|im_end|>\n<|im_start|>assistant\nКонечно, с удовольствием помогу вам! Счётчики - это инструмент, который позволяет сравнить ожидаемый результат с фактическим. Они используются в тестировании программного обеспечения и различных алгоритмах. Создание корректных счётчиков поможет вам быстро обнаружить ошибки в вашем коде.<|im_end|>\n<|im_start|>user\nКак мне создать счётчик?<|im_end|>\n<|im_start|>assistant\nВы можете создать простой счётчик, который будет работать с числами. Например, в Python вы можете создать переменную и задать ей начальное значение 0. Далее, внутри цикла или иного блока кода, вы можете увеличивать это значение на 1 каждый раз, когда выполняется определенное условие. В конце выполнения программы вы можете сравнить значение этой переменной с ожидаемым результато

#  Токенизация данных

In [37]:
# Инициализация токенизатора
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Функция токенизации
def tokenize_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True, max_length=2048)

# Преобразование ваших данных в формат датасета и токенизация
from datasets import Dataset

# Создаем Dataset из formatted_samples
formatted_dataset = Dataset.from_dict({"text": formatted_samples})

# Применяем токенизацию
tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)

# Разделение на тренировочную и валидационную выборку
train_size = int(0.95 * len(tokenized_dataset))
train_dataset = tokenized_dataset.select(range(train_size))
eval_dataset = tokenized_dataset.select(range(train_size, len(tokenized_dataset)))


Map: 100%|██████████| 37731/37731 [00:25<00:00, 1497.24 examples/s]


In [46]:
train_dataset
# получаем датасет (везде в начале стоит системное сообщение, поэтому в начале все данные одинаковые)

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 35844
})

# Загрузка модели + квантизация

In [39]:
# Настройка квантизации для экономии памяти
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Загрузка модели с квантизацией
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]C:\python\sirius_2025\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Сергей\.cache\huggingface\hub\models--Qwen--Qwen1.5-7B-Chat. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 4 files: 100%|██████████| 4/4 [09:07<00:00, 136.98s/it]

# LoRA

In [40]:
# Настройка LoRA для эффективного дообучения
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj", 
        "v_proj", 
        "o_proj",
        "gate_proj", 
        "up_proj", 
        "down_proj",
    ]
)

# Применение LoRA к модели
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 39,976,960 || all params: 7,761,301,504 || trainable%: 0.5151


# Подготовка trainer к обучанию

In [41]:
# Настройка аргументов обучения
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    logging_steps=10,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    save_strategy="steps",
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    do_eval=True,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    optim="adamw_torch",
    report_to="none",
    fp16=True,
    remove_unused_columns=False,
    group_by_length=True,
)

# Инициализация SFT тренера
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

C:\python\sirius_2025\.venv\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Truncating eval dataset: 100%|██████████| 1887/1887 [00:00<00:00, 47293.16 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


# Обучаем

#### К сожелению на данном этапе мне пришлось остановится, т.к. обучалось слишком долго =(
#### Но я считаю что проделал достойную работу, как минимум я получил ценный опыт

In [ ]:
# Запуск обучения
trainer.train()

In [ ]:
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Training completed and model saved!")